In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow import keras
import random
import cv2
import math
from PIL import Image
import seaborn as sns
import csv
import json

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [9]:
DenseNet121

<function keras.src.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax')>

In [11]:
# Define Model Architechture
model_d = DenseNet121(weights=None,include_top=False, input_shape=(32, 32, 3)) 
x = model_d.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x) 
x = Dense(512, activation='relu')(x) 
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

preds = Dense(3, activation='softmax')(x) #FC-layer

model = Model(inputs=model_d.input,outputs=preds)
# model.summary()


for layer in model.layers[:-8]:
    layer.trainable=False

for layer in model.layers[-8:]:
    layer.trainable=True

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
# Define constants
EPOCHS = 10
CHECKPOINT_NAME = 'checkpoints'

label_dict = {'fivefingerR': 0, 'fourfingerR': 1, 'threefingerR': 2, 'twofingerR': 3, 'onefingerR': 4, 
              'fistR': 5, 'cowabungaR': 6, 'spidermanR': 7, 'okR': 8, 'cR': 9, 'thumbR': 10, 
              'lR': 11, 'oR': 12, 'rockonR': 13, 'palmR': 14, 'sidehandR': 15}

with open('data_pad.json', 'r') as file:
    raw_data = json.load(file)

labels = np.array([])
data = np.array([])

for label, arrays in raw_data.items():
    for array in arrays:
        labels = np.append(labels, label_dict[label])
        normalizedData = (array-np.min(array))/(np.max(array)-np.min(array))
        data = np.append(data, normalizedData)
        
# print(data)
# print(labels)
# mlb = LabelBinarizer()
# labels = mlb.fit_transform(labels)
# print(labels[1])

(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.4)
print(xtrain.shape, xtest.shape)

# anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
# checkpoint = ModelCheckpoint(CHECKPOINT_NAME, verbose=1, save_best_only=True)
# datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)
# datagen.fit(xtrain)

# Fits-the-model
# history = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=128),
#                steps_per_epoch=xtrain.shape[0] //128,
#                epochs=EPOCHS,
#                verbose=2,
#                validation_data=(xtrain, ytrain))

model_t = model.fit(xtrain, ytrain)

In [5]:
# Load the image
image_path = "data_pad.jpg"
image = load_img(image_path, target_size=(32, 32))  # Resize the image to match the input size of the CNN
image_array = img_to_array(image)  # Convert the image to a numpy array
expanded_image_array = tf.expand_dims(image_array, axis=0)  # Expand the dimensions to match the batch size

# Preprocess the image
preprocessed_image = preprocess_input(expanded_image_array)

# Load the pre-trained CNN model
model = model_d

# Make predictions on the image
predictions = model.predict(preprocessed_image)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/image.jpg'